In [2]:
from urllib import request
import json
import time
import csv
import pandas as pd
from datetime import datetime
from datetime import timedelta

def get_data(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.89 Safari/537.36'
    }
    req = request.Request(url,headers = headers)
    response = request.urlopen(req)
    if response.getcode() == 200:
        return response.read()
    return None

def parse_data(html):
    data = json.loads(html)['cmts']
    comments = []
    for item in data:
        comment = {
            'id':item['id'],
            'nickName':item['nickName'],
            'cityName':item['cityName'] if 'cityName' in item else'',#处理cityName不存在的情况
            'content':item['content'].replace('\n','',10), #处理评论内容的换情况
            'score':item['score'],
            'startTime':item['startTime']
        }
        print(comment)
        comments.append(comment)
    return comments


def save_to_txt():
    start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(start_time)
    end_time = '2019-10-10 00:00:00'
    df = pd.DataFrame(columns=['id','content','score'])
    while start_time > end_time:
        url = 'http://m.maoyan.com/mmdb/comments/movie/1277939.json?_v_==yes&offset=0&startTime=' + start_time.replace(' ','%20')
        html = None
        """
            问题：当请求过于频繁时，服务器会拒绝连接，实际上是服务器的反爬虫策略
            解决：1.在每个请求增加延时0.1秒，尽量减少请求别拒绝
                 2.如果被拒绝，则0.5秒后重试
        """
        try:
            html = get_data(url)
        except Exception as e:
            time.sleep(0.5)
            html = get_data(url)
        else:
            time.sleep(0.1)      
        comments = parse_data(html)            
        start_time = comments[14]['startTime']
        start_time = datetime.strptime(start_time,'%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1) #转换为datetime类型，减1秒，避免获取到重复数据
        start_time = datetime.strftime(start_time,'%Y-%m-%d %H:%M:%S') #转换为str     
        for item in comments:
            item_list = []
            item_list.append(item['id'])
            item_list.append(item['content'])
            item_list.append(item['score'])
            item_list.append(item['startTime'])
            with open("comments.csv", "a", newline='',encoding="utf_8_sig") as file:
                writer = csv.writer(file,delimiter=',')
                writer.writerow(item_list)
                
if __name__ == "__main__":
    html = get_data('http://m.maoyan.com/mmdb/comments/movie/1277939.json')
    comments = parse_data(html)
    print(comments)
    save_to_txt()

{'id': 1137059900, 'nickName': '爱李不理李', 'cityName': '亳州', 'content': '不错这个电影', 'score': 5.0, 'startTime': '2021-06-04 13:13:04'}
{'id': 1137064071, 'nickName': 'FdK115888991', 'cityName': '郑州', 'content': '因为你我相信真爱，因为你我相信永远', 'score': 3.5, 'startTime': '2021-06-04 13:03:30'}
{'id': 1137063175, 'nickName': '郑晓丹521', 'cityName': '南阳', 'content': '很不错的，爱国', 'score': 5.0, 'startTime': '2021-06-04 13:02:45'}
{'id': 1137060457, 'nickName': '朵宝宝0720', 'cityName': '昆明', 'content': '特别好看，，', 'score': 5.0, 'startTime': '2021-06-04 13:01:13'}
{'id': 1137058857, 'nickName': '保俊平安', 'cityName': '深圳', 'content': '扣扣扣同一时间用了', 'score': 4.0, 'startTime': '2021-06-04 12:31:05'}
{'id': 1137059599, 'nickName': 'nDQ777446289', 'cityName': '上海', 'content': '好看，喜欢东东小可爱', 'score': 4.5, 'startTime': '2021-06-04 12:31:01'}
{'id': 1137057951, 'nickName': 'roi434462279', 'cityName': '武汉', 'content': '超级好看！！', 'score': 5.0, 'startTime': '2021-06-04 12:27:57'}
{'id': 1137054853, 'nickName': 'S光子', 'cityName': '忻州',

JSONDecodeError: Expecting value: line 1 column 1 (char 0)